# UK Loyalty Indicator

In [2]:
import pandas as pd
import datetime 
from google.cloud import bigquery

from pyspark.sql.functions import *
from pyspark.sql.functions import col,udf,array
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors, VectorUDT

from pyspark.sql.functions import col,udf,array
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors, VectorUDT
import datetime

In [2]:
import datetime;print(datetime.datetime.now())

2018-02-15 09:36:19.977787


In [3]:
# Exporting BQ Tables to Google Storage

bigquery_client = bigquery.Client()
dataset_ref = bigquery_client.dataset("tom") # DATASET OF BQ
table_ref = dataset_ref.table("Yocuda_clean_data_Nov15_Nov17_20171214_v01") # TABLE NAME OF BQ


job = bigquery_client.extract_table(table_ref, 'gs://westfield-tom/datalab/Yocuda_clean_data_Nov15_Nov17_20171214_v01_*.csv') # GOOGLE CLOUD BUCKET
print(datetime.datetime.now())

2018-02-15 09:36:20.670759


In [4]:
print(datetime.datetime.now())

2018-02-15 09:36:20.676615


In [5]:
#Importing 
data = (spark.read.format("csv")
      .option("header", "true")
      .load('gs://westfield-tom/datalab/Yocuda_clean_data_Nov15_Nov17_20171214_v01_*.csv'))
print(datetime.datetime.now())

2018-02-15 09:36:53.112431


In [6]:
print(datetime.datetime.now())

2018-02-15 09:36:53.117959


In [7]:
# Overall - list of unique indicators
from pyspark.sql.functions import *

overall = data.filter((to_date(col("timestamp"))>'2016-11-30') & (to_date(col("timestamp"))<'2017-12-01') & (col("retailer_name")=='Argos')).select("identifier").distinct()
overall.show(5)
overall.count()
print(datetime.datetime.now())

+--------------------+
|          identifier|
+--------------------+
|sazidchoudhury@gm...|
|shakur_b9@hotmail...|
|michaelharteis@ho...|
|  rebekah869@msn.com|
|     skorp@gmail.com|
+--------------------+
only showing top 5 rows

2018-02-15 09:39:02.225298


In [8]:
print(datetime.datetime.now())

2018-02-15 09:39:02.231004


In [9]:
# Brand

df1 = data.filter((to_date(col("timestamp"))>'2016-11-30') & \
                  (to_date(col("timestamp"))<'2017-12-01') & \
                  (col("brandName").isNotNull()) & \
                  (col("identifier").isNotNull()) & \
                  (col("item_total")>0) & \
                  (col("retailer_name")=='Argos'))\
.groupBy("identifier","brandName").agg(sum("item_total").alias("spend"))


df2 = data.filter((to_date(col("timestamp"))>'2016-11-30') & \
                  (to_date(col("timestamp"))<'2017-12-01') & \
                  (col("brandName").isNotNull()) & \
                  (col("identifier").isNotNull()) & \
                  (col("item_total")>0) & \
                  (col("retailer_name")=='Argos'))\
.groupBy("identifier").agg(sum("item_total").alias("spend_overall"))

df3 = df1.join(df2,["identifier"],"left")

brand = df3.withColumn('entropy', -(col('spend')/ col('spend_overall'))*log10(col('spend')/col('spend_overall')))\
.groupBy(col("identifier")).agg(sum("entropy").alias("brand_diversity"))

brand.show(5)
brand.count()
print(datetime.datetime.now())

+--------------------+-------------------+
|          identifier|    brand_diversity|
+--------------------+-------------------+
|   -.tmh24@gmail.com|                0.0|
|-indiaismial78672...|                0.0|
|.j.douglas@hotmai...|                0.0|
|    00673@uk.mcd.com|                0.0|
|00blackswan7@yaho...|0.22891523252096496|
+--------------------+-------------------+
only showing top 5 rows

2018-02-15 09:42:49.232262


In [10]:
print(datetime.datetime.now())

2018-02-15 09:42:49.238452


In [11]:
# category

df1 = data.filter((to_date(col("timestamp"))>'2016-11-30') & \
                  (to_date(col("timestamp"))<'2017-12-01') & \
                  (col("cat2").isNotNull()) & \
                  (col("identifier").isNotNull()) & \
                  (col("item_total")>0) & \
                  (col("retailer_name")=='Argos'))\
.groupBy("identifier","cat2").agg(sum("item_total").alias("spend"))


df2 = data.filter((to_date(col("timestamp"))>'2016-11-30') & \
                  (to_date(col("timestamp"))<'2017-12-01') & \
                  (col("cat2").isNotNull()) & \
                  (col("identifier").isNotNull()) & \
                  (col("item_total")>0) & \
                  (col("retailer_name")=='Argos'))\
.groupBy("identifier").agg(sum("item_total").alias("spend_overall"))

df3 = df1.join(df2,["identifier"],"left")

category = df3.withColumn('entropy', -(col('spend')/ col('spend_overall'))*log10(col('spend')/col('spend_overall')))\
.groupBy(col("identifier")).agg(sum("entropy").alias("category_diversity"))

category.show(5)
category.count()
print(datetime.datetime.now())

+--------------------+-------------------+
|          identifier| category_diversity|
+--------------------+-------------------+
|   -.tmh24@gmail.com|                0.0|
|-indiaismial78672...|  0.299594933046483|
|.j.douglas@hotmai...|                0.0|
|    00673@uk.mcd.com| 0.2863175935284513|
|00blackswan7@yaho...|0.22891523252096496|
+--------------------+-------------------+
only showing top 5 rows

2018-02-15 09:46:29.882627


In [12]:
print(datetime.datetime.now())

2018-02-15 09:46:29.889011


In [13]:
data.registerTempTable("data")

xx1 =data.selectExpr("identifier","month(timestamp) as month", "year(timestamp) as year", "item_total", "1 as ind").filter((to_date(col("timestamp"))>'2016-11-30') & \
                  (to_date(col("timestamp"))<'2017-12-01') & \
                  (col("identifier").isNotNull()) & \
                  (col("item_total")>0) & \
                  (col("retailer_name")=='Argos'))

xx2 =data.selectExpr("timestamp ", "1 as ind").filter((to_date(col("timestamp"))>'2016-11-30') & \
                  (to_date(col("timestamp"))<'2017-12-01') & \
                  (col("identifier").isNotNull()) & \
                  (col("item_total")>0) & \
                  (col("retailer_name")=='Argos'))\
.groupBy(col("ind")).agg(max("timestamp").alias("timestamp_mx"))

xx3 = xx1.join(xx2,["ind"],"left")

xx4 = xx3.selectExpr("identifier","((month(timestamp_mx) - month)+ (year(timestamp_mx)- year)) as recency", "item_total")\
.groupBy(col("identifier"),col("recency")).agg(sum("item_total").alias("spend"))


timeseries = xx4.selectExpr("identifier", "((exp((-1) * recency)) * log10(spend)) as timeseries")\
.groupBy(col("identifier")).agg(sum("timeseries").alias("timeseries"))

timeseries.show(5)
timeseries.count()
print(datetime.datetime.now())

+--------------------+-------------------+
|          identifier|         timeseries|
+--------------------+-------------------+
|andycooksupply@ho...|0.05852528701915769|
|judyfashions5@gma...| 1.7594520520367305|
|michellearnold600...|0.22629717562804758|
|pspearpoint@yahoo...| 0.6085991155504794|
|typeitright@bluey...| 2.5440432268136592|
+--------------------+-------------------+
only showing top 5 rows

2018-02-15 09:53:17.592476


In [14]:
print(datetime.datetime.now())

2018-02-15 09:53:17.599171


In [15]:
combined = overall.join(brand,["identifier"],"left").join(category,["identifier"],"left").join(timeseries,["identifier"],"left")

combined.show(5)
combined.count()
print(datetime.datetime.now())

+--------------------+-------------------+-------------------+--------------------+
|          identifier|    brand_diversity| category_diversity|          timeseries|
+--------------------+-------------------+-------------------+--------------------+
|   -.tmh24@gmail.com|                0.0|                0.0|0.002840198153859...|
|-indiaismial78672...|                0.0|  0.299594933046483|8.485499381124429E-5|
|.j.douglas@hotmai...|                0.0|                0.0|0.021535558112033094|
|    00673@uk.mcd.com|                0.0| 0.2863175935284513|2.288245512950547E-4|
|00blackswan7@yaho...|0.22891523252096496|0.22891523252096496|1.813540983064614...|
+--------------------+-------------------+-------------------+--------------------+
only showing top 5 rows

2018-02-15 10:06:45.716609


In [16]:
print(datetime.datetime.now())

2018-02-15 10:06:45.722780


In [17]:
# Importing the data and starting the process
data_new = combined

# Replacing the missing values by 0 
# Although it's not necessary as there are no missing values most of the time
data_new = data_new.fillna(0)

# Creating spark sql tables for faster extraction of the percentiles
data_new.createOrReplaceTempView('temporary_intermediate_table')

# Storing the column names in a set (ordered list of sorts)
col_names = set(data_new.columns) - {"identifier"}

# Writing queries to calculate the 99th (upper limit) and 1st (lower limit) percentile
upper_limit_query = 'SELECT ' + ', '.join(['Percentile({var_name}, 0.99) AS {var_name}'.format(var_name = i) for i in col_names]) + ' from ' 
lower_limit_query = 'SELECT ' + ', '.join(['Percentile({var_name}, 0.01) AS {var_name}'.format(var_name = i) for i in col_names]) + ' from ' 

import pandas as pd

# Saving the capping limits in two pandas DataFrames 
upper_limit = spark.sql(upper_limit_query + 'temporary_intermediate_table').toPandas()
lower_limit = spark.sql(lower_limit_query + 'temporary_intermediate_table').toPandas()

# Deciding which columns need outlier treatment (Dropping those which have 1st and 99th percentile as same)
columns_to_treat = {var_name for var_name in col_names if upper_limit.iloc[0][var_name] > lower_limit.iloc[0][var_name]}

# Writing a SQL query for outlier treatment
outlier_treated_data = spark.sql('SELECT identifier, '+ ', '.join(['IF({var_name} >= {maximum}, {maximum}, IF({var_name} <={minimum}, {minimum}, {var_name})) AS  {var_name}'.format(var_name = var_name, maximum = upper_limit.iloc[0][var_name], minimum = lower_limit.iloc[0][var_name])  for var_name in columns_to_treat]) + ' FROM ' + 'temporary_intermediate_table')

outlier_treated_data.show(5)
outlier_treated_data.count()
print(datetime.datetime.now())

+--------------------+-------------------+-------------------+--------------------+
|          identifier|    brand_diversity| category_diversity|          timeseries|
+--------------------+-------------------+-------------------+--------------------+
|   -.tmh24@gmail.com|                0.0|                0.0|0.002840198153859...|
|-indiaismial78672...|                0.0|  0.299594933046483|8.485499381124429E-5|
|.j.douglas@hotmai...|                0.0|                0.0|0.021535558112033094|
|    00673@uk.mcd.com|                0.0| 0.2863175935284513|2.288245512950547E-4|
|00blackswan7@yaho...|0.22891523252096496|0.22891523252096496|1.813540983064614...|
+--------------------+-------------------+-------------------+--------------------+
only showing top 5 rows

2018-02-15 10:34:35.313893


In [22]:
print(datetime.datetime.now())

2018-02-15 10:42:04.141213


In [24]:
#Saving data into GCS
(outlier_treated_data.coalesce(1).write.option("header", "true").mode("overwrite").csv("gs://westfield-tom/datalab/Loyalty_indicator/outlier_treated_data"))
print(datetime.datetime.now())

2018-02-15 10:57:10.178770


In [3]:
print(datetime.datetime.now())

2018-02-20 12:26:35.490653


In [13]:
# Reading data in from GCS
outlier_treated_data = (spark.read.format("csv")
      .option("header", "true").option("inferSchema", "true")
      .load('gs://westfield-tom/datalab/Loyalty_indicator/outlier_treated_data/part-00000-9896de6e-3eeb-4178-bf6c-a6c368cdeec4-c000.csv'))

outlier_treated_data.show(5)
outlier_treated_data.count()

print(datetime.datetime.now())

+--------------------+-------------------+-------------------+--------------------+
|          identifier|    brand_diversity| category_diversity|          timeseries|
+--------------------+-------------------+-------------------+--------------------+
|   -.tmh24@gmail.com|                0.0|                0.0|0.002840198153859...|
|-indiaismial78672...|                0.0|  0.299594933046483|8.485499381124429E-5|
|.j.douglas@hotmai...|                0.0|                0.0|0.021535558112033094|
|    00673@uk.mcd.com|                0.0| 0.2863175935284513|2.288245512950547E-4|
|00blackswan7@yaho...|0.22891523252096496|0.22891523252096496|1.813540983064614...|
+--------------------+-------------------+-------------------+--------------------+
only showing top 5 rows

2018-02-20 12:46:40.974361


In [14]:
outlier_treated_data.dtypes

[('identifier', 'string'),
 ('brand_diversity', 'double'),
 ('category_diversity', 'double'),
 ('timeseries', 'double')]

In [15]:
print(datetime.datetime.now())

2018-02-20 12:47:04.721610


In [16]:
# NORMALIZATION: TARGET is at 0th INDEX
from pyspark.sql.functions import col,udf,array
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors, VectorUDT

# NORMALIZATION: TARGET is at 0th INDEX
from pyspark.sql.functions import col,udf,array
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors, VectorUDT


# Defining the function to normalize the data.
def normalization_missing_value(data_new):
  i = 0
#   Transforming the variables as per Maximum and Minimum Value
  for cols in data_new.columns[1:]:
    maxs = data_new.agg(max(col(cols)).alias("max")).toPandas()
    mins = data_new.agg(min(col(cols)).alias("min")).toPandas()
    data_new = data_new.withColumn(cols,((col(cols)-mins["min"][0])/(maxs["max"][0]-mins["min"][0])))
    i = i+1
  return data_new
                           
final= normalization_missing_value(outlier_treated_data)
final.show(5)
print(datetime.datetime.now())

+--------------------+-------------------+-------------------+--------------------+
|          identifier|    brand_diversity| category_diversity|          timeseries|
+--------------------+-------------------+-------------------+--------------------+
|   -.tmh24@gmail.com|                0.0|                0.0|9.893906597435384E-4|
|-indiaismial78672...|                0.0|0.34327102441366086|2.955946513638719E-5|
|.j.douglas@hotmai...|                0.0|                0.0|0.007501969543729179|
|    00673@uk.mcd.com|                0.0| 0.3280580637287231|7.971164739463231E-5|
|00blackswan7@yaho...|0.34604067677403916| 0.2622873677211728|6.317518752232143E-5|
+--------------------+-------------------+-------------------+--------------------+
only showing top 5 rows

2018-02-20 12:47:29.878432


In [17]:
print(datetime.datetime.now())

2018-02-20 12:50:09.799126


In [18]:
#Saving data into GCS
(final.coalesce(1).write.option("header", "true").mode("overwrite").csv("gs://westfield-tom/datalab/Loyalty_indicator/normalised_metrics"))
print(datetime.datetime.now())

2018-02-20 12:51:25.404590


In [27]:
print(datetime.datetime.now())

2018-02-15 11:42:54.105759


In [28]:
################################################ Defining Target Variable ############################################

# Spend Based Model 
from pyspark.sql.window import Window
df1 = data.filter((to_date(col("timestamp"))>'2016-11-30') & \
                  (to_date(col("timestamp"))<'2017-12-01') & \
                  (col("identifier").isNotNull()) & \
                  (col("item_total")>0))\
.groupBy("identifier").agg(sum("item_total").alias("spend"))

df2 = df1.withColumn('percentile', percent_rank().over(Window.orderBy(desc('spend')))) \
.select('identifier','spend', 'percentile')

df3 = df2.withColumn("bucket", (when(col("percentile")<=0.10,"0-10").otherwise(\
      when(col('percentile')<=0.20,"10-20").otherwise(\
        when(col('percentile')<=0.30,"20-30").otherwise(\
          when(col('percentile')<=0.40,"30-40").otherwise(\
            when(col('percentile')<=0.50,"40-50").otherwise(\
              when(col('percentile')<=0.60,"50-60").otherwise(\
                when(col('percentile')<=0.70,"60-70").otherwise(\
                  when(col('percentile')<=0.80,"70-80").otherwise(\
                    when(col('percentile')<=0.90,"80-90").otherwise('90-100')))))))))))\

output = df3.groupBy("bucket").agg((sum(col("spend")).alias("spend"))\
                       ,(countDistinct(col("identifier")).alias("cnt_shp"))\
                       ,(avg(col("spend")).alias("avg_spend")))

output.show()
print(datetime.datetime.now())
######################################################################################################################

+------+--------------------+-------+------------------+
|bucket|               spend|cnt_shp|         avg_spend|
+------+--------------------+-------+------------------+
|  0-10|1.1849354187978327E9|2106163| 562.6038529771118|
| 10-20|4.2566265366426456E8|2131922|199.66145743806038|
| 20-30|2.5298825609055772E8|2081822|121.52252022053649|
| 30-40| 1.754814611097583E8|2122119| 82.69162149236602|
| 40-50|1.2177078294859919E8|2095970| 58.09757913930027|
| 50-60| 8.995130797959337E7|2099623|42.841647276484096|
| 60-70| 6.702846659015419E7|2104122| 31.85578906078364|
| 70-80| 5.454329904064297E7|2382295|22.895274951524883|
| 80-90|3.0570063189468637E7|1833997|16.668545907909685|
|90-100|2.1033439779639587E7|2099218|10.019654833199594|
+------+--------------------+-------+------------------+

2018-02-15 11:45:15.832439


In [29]:
print(datetime.datetime.now())

2018-02-15 11:45:15.838643


In [30]:
################################################ Defining Target Variable ############################################

# Spend Based Model 
from pyspark.sql.window import Window
df1 = data.filter((to_date(col("timestamp"))>'2016-11-30') & \
                  (to_date(col("timestamp"))<'2017-12-01') & \
                  (col("identifier").isNotNull()) & \
                  (col("item_total")>0))\
.groupBy("identifier").agg(sum("item_total").alias("spend"))

df2 = df1.withColumn('percentile', percent_rank().over(Window.orderBy(desc('spend')))) \
.select('identifier','spend', 'percentile')

target = df2.withColumn("target",(when(col("percentile")<=0.20,1).otherwise(0)))\
                     .select('identifier','target')

final_w_target = final.join(target,["identifier"],"left").fillna(0)

final_w_target.show(5)
print(datetime.datetime.now())

+--------------------+-------------------+-------------------+--------------------+------+
|          identifier|    brand_diversity| category_diversity|          timeseries|target|
+--------------------+-------------------+-------------------+--------------------+------+
|   -.tmh24@gmail.com|                0.0|                0.0|9.893906597435384E-4|     0|
|-indiaismial78672...|                0.0|0.34327102441366086|2.955946513638719E-5|     0|
|.j.douglas@hotmai...|                0.0|                0.0|0.007501969543729179|     0|
|    00673@uk.mcd.com|                0.0| 0.3280580637287231|7.971164739463231E-5|     0|
|00blackswan7@yaho...|0.34604067677403916| 0.2622873677211728|6.317518752232143E-5|     0|
+--------------------+-------------------+-------------------+--------------------+------+
only showing top 5 rows

2018-02-15 11:52:45.428069


In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import numpy as np

In [32]:
print(datetime.datetime.now())

2018-02-15 11:52:45.496895


In [33]:
from pyspark.sql.functions import col
import random
random.seed(7)
train, test = final_w_target.randomSplit([0.8, 0.2])

train.show(5)
print(datetime.datetime.now())

+--------------------+-------------------+------------------+--------------------+------+
|          identifier|    brand_diversity|category_diversity|          timeseries|target|
+--------------------+-------------------+------------------+--------------------+------+
|.j.douglas@hotmai...|                0.0|               0.0|0.007501969543729179|     0|
|    00673@uk.mcd.com|                0.0|0.3280580637287231|7.971164739463231E-5|     0|
|00blackswan7@yaho...|0.34604067677403916|0.2622873677211728|6.317518752232143E-5|     0|
| 01388dave@gmail.com|                0.0|               0.0|  0.7242505480831564|     0|
|01656858872@talkt...| 0.8913519382514233|0.8211887484951836|  0.7681917813342942|     1|
+--------------------+-------------------+------------------+--------------------+------+
only showing top 5 rows

2018-02-15 12:00:22.758377


In [34]:
print(datetime.datetime.now())

2018-02-15 12:00:22.764122


In [35]:
####### Saving Test and train dataset ############

train.write.mode("overwrite").parquet("gs://westfield-tom/datalab/train_set2")
test.write.mode("overwrite").parquet("gs://westfield-tom/datalab/test_set2")
print(datetime.datetime.now())

2018-02-15 12:16:37.918147


In [36]:
######################################################################################################################
################################################## RUN Data Proc Code ################################################
######################################################################################################################

In [37]:
################ To Run After training the model  ##################

In [38]:
# Coeffecients and Intercept for Logistic Regression ##

In [41]:
print(datetime.datetime.now())

2018-02-15 13:40:00.913915


In [19]:
from pyspark.ml.classification import *
print ('Reading Saved Model...')

lrModel = LogisticRegressionModel.read().load("gs://westfield-tom/datalab/lr_model1")

# Print the coefficients and intercept for logistic regression
print ('Model Coeffecients...')
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

Reading Saved Model...
Model Coeffecients...
Coefficients: [0.9813838992695821,1.531667952505228,1.5192042104267771]
Intercept: -1.97776262668


In [43]:
print(datetime.datetime.now())

2018-02-15 13:40:03.140895


In [44]:
print ('Reading Saved Data...')
train_predictions = sqlContext.read.parquet("gs://westfield-tom/datalab/train_predictions")
print ('Sample Records...')
train_predictions.select("prediction", "features", "probability", "label").show(5)
print(datetime.datetime.now())

Reading Saved Data...
Sample Records...
+----------+--------------------+--------------------+-----+
|prediction|            features|         probability|label|
+----------+--------------------+--------------------+-----+
|       0.0|[0.0,0.0,0.001062...|[0.87827002886590...|    0|
|       0.0|[0.40762356689962...|[0.67405520613893...|    0|
|       0.0|[0.0,0.0,7.440795...|[0.87832169649086...|    1|
|       0.0|[0.0,0.0,0.526575...|[0.76454965013404...|    0|
|       0.0|[0.0,0.0,0.066700...|[0.86720069076097...|    0|
+----------+--------------------+--------------------+-----+
only showing top 5 rows

2018-02-15 13:40:04.332413


In [45]:
print(datetime.datetime.now())

2018-02-15 13:40:04.337918


In [46]:
#Confusion matrix
#Logic for getting confusion matrix and model evaluation parameters for Cross Validation Data
TP = train_predictions.filter((col("prediction") == 1) &  (col("label")==1)).count()
FP = train_predictions.filter((col("prediction") == 1) &  (col("label") == 0)).count()
TN = train_predictions.filter((col("prediction") == 0) &  (col("label") == 0)).count()
FN = train_predictions.filter((col("prediction") == 0) &  (col("label") == 1)).count()

print 'True Positive = ', TP
print 'False Negative = ', FN
print 'True Negative = ', TN
print 'False Positive = ', FP
print(datetime.datetime.now())

True Positive =  715775
False Negative =  1766430
True Negative =  7046137
False Positive =  335019
2018-02-15 13:40:30.455939


In [47]:
print(datetime.datetime.now())

2018-02-15 13:40:30.461987


In [48]:
# KS For Cross Validation
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import col
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType


outlier_treated_cv=train

split1_udf = udf(lambda value: value[0].item(), FloatType())
split2_udf = udf(lambda value: value[1].item(), FloatType())
prediction_prob = train_predictions.select(split1_udf('probability').alias('Prob_0'), split2_udf('probability').alias('Prob_1'),"label")


# Size for each bins
bin_size = float(outlier_treated_cv.count()/10)
# Total Events
total_events = float(str(outlier_treated_cv.select(sum("target")).head()).replace(")","").split("=")[1])
# Total non events
total_non_events = outlier_treated_cv.count() - float(str(outlier_treated_cv.select(sum("Target")).head()).replace(")","").split("=")[1])

# Decile Tables used for KS
x = prediction_prob.withColumn("row_number",ceil(F.row_number().over(Window.partitionBy().orderBy("prob_1"))/bin_size)).groupBy("row_number").agg(min("prob_1").alias("minimum"),max("prob_1").alias("maximum"),sum("label").alias("number_of_target"), (sum("label")/int(bin_size)).alias("perc_of_target"),(int(bin_size)-sum("label")).alias("number_of_non_target"))
x= x.filter(col("row_number")<=10).withColumn("cum_tar", sum("number_of_target").over(Window.partitionBy().orderBy("row_number"))).withColumn("cum_non_tar", sum("number_of_non_target").over(Window.partitionBy().orderBy("row_number"))).withColumn("per_dist_target",col("cum_tar")/total_events).withColumn("per_dist_non_target",col("cum_non_tar")/total_non_events).withColumn("spread",abs(col("per_dist_target")-col("per_dist_non_target")))
x.show()
print(datetime.datetime.now())

+----------+----------+----------+----------------+-------------------+--------------------+-------+-----------+--------------------+-------------------+--------------------+
|row_number|   minimum|   maximum|number_of_target|     perc_of_target|number_of_non_target|cum_tar|cum_non_tar|     per_dist_target|per_dist_non_target|              spread|
+----------+----------+----------+----------------+-------------------+--------------------+-------+-----------+--------------------+-------------------+--------------------+
|         1|0.12155755|0.12156872|           82306| 0.0834462089997729|              904030|  82306|     904030|0.033158488437676256|0.12247813824233988| 0.08931964980466363|
|         2|0.12156872| 0.1216542|          130398|0.13220444148849886|              855938| 212704|    1759968| 0.08569172508258803|0.23844076414067503|   0.152749039058087|
|         3| 0.1216542|0.12251811|          171446| 0.1738210913927911|              814890| 384150|    2574858| 0.1547619047

In [ ]:
######################################################################################################################
# Scoring Loyalty Indicator

In [22]:
final.show(5)

+--------------------+-------------------+-------------------+--------------------+
|          identifier|    brand_diversity| category_diversity|          timeseries|
+--------------------+-------------------+-------------------+--------------------+
|   -.tmh24@gmail.com|                0.0|                0.0|9.893906597435384E-4|
|-indiaismial78672...|                0.0|0.34327102441366086|2.955946513638719E-5|
|.j.douglas@hotmai...|                0.0|                0.0|0.007501969543729179|
|    00673@uk.mcd.com|                0.0| 0.3280580637287231|7.971164739463231E-5|
|00blackswan7@yaho...|0.34604067677403916| 0.2622873677211728|6.317518752232143E-5|
+--------------------+-------------------+-------------------+--------------------+
only showing top 5 rows



In [ ]:
# brand_diversity = 0.243383328
# category_diversity = 0.379853841
# timeseries = 0.376762831

In [35]:
loyalty_score=final.withColumn("brand_diversity_calc", when((col("brand_diversity").isNotNull()),col("brand_diversity")*0.243383328).otherwise(0)) \
                    .withColumn("category_diversity_calc", when((col("category_diversity").isNotNull()),col("category_diversity")*0.379853841).otherwise(0)) \
                    .withColumn("timeseries_calc", when((col("timeseries").isNotNull()),col("timeseries")*0.376762831).otherwise(0)) \
                    .withColumn("loyalty_score", col("brand_diversity_calc")+col("category_diversity_calc")+col("timeseries_calc")) 
loyalty_score.show(5)

+--------------------+-------------------+-------------------+--------------------+--------------------+-----------------------+--------------------+--------------------+
|          identifier|    brand_diversity| category_diversity|          timeseries|brand_diversity_calc|category_diversity_calc|     timeseries_calc|       loyalty_score|
+--------------------+-------------------+-------------------+--------------------+--------------------+-----------------------+--------------------+--------------------+
|   -.tmh24@gmail.com|                0.0|                0.0|9.893906597435384E-4|                 0.0|                    0.0|3.727656259299332...|3.727656259299332...|
|-indiaismial78672...|                0.0|0.34327102441366086|2.955946513638719E-5|                 0.0|    0.13039281712753387|1.113690776763104E-5|  0.1304039540353015|
|.j.douglas@hotmai...|                0.0|                0.0|0.007501969543729179|                 0.0|                    0.0|0.002826463283371

In [45]:
loyalty_score_band=loyalty_score.selectExpr('identifier', 'loyalty_score').withColumn("loyalty_tier", when((col("loyalty_score")<=0.205438),"5.Occasional") \
                                .when((col("loyalty_score")<=0.383549),"4.Bronze") \
                                .when((col("loyalty_score")<=0.544249),"3.Silver") \
                                .when((col("loyalty_score")<=0.673125),"2.Gold") \
                                .when((col("loyalty_score")>0.673125),"1.Platinum"))
loyalty_score_band.show(10)

+--------------------+--------------------+------------+
|          identifier|       loyalty_score|loyalty_tier|
+--------------------+--------------------+------------+
|   -.tmh24@gmail.com|3.727656259299332...|5.Occasional|
|-indiaismial78672...|  0.1304039540353015|5.Occasional|
|.j.douglas@hotmai...|0.002826463283371...|5.Occasional|
|    00673@uk.mcd.com| 0.12464414796431433|5.Occasional|
|00blackswan7@yaho...| 0.18387519767380472|5.Occasional|
| 01388dave@gmail.com| 0.27287068684911164|    4.Bronze|
|01555664963@talkt...|  0.7426570751179802|  1.Platinum|
|01656858872@talkt...|   0.818298011739202|  1.Platinum|
|       01ian@live.uk| 0.10694529543243954|5.Occasional|
|01tinalou@btinter...|1.332780514470631E-4|5.Occasional|
+--------------------+--------------------+------------+
only showing top 10 rows



In [ ]:
######################################################################################################################
# Saving Loyalty Indicator to GCS

In [60]:
#Saving data into GCS
(loyalty_score_band.coalesce(1).write.option("header", "true").mode("overwrite").csv("gs://westfield-tom/datalab/Loyalty_indicator/loyalty_score"))
print(datetime.datetime.now())

2018-02-20 15:26:09.306517


In [ ]:
print(datetime.datetime.now())

In [ ]:
# Reading data in from GCS
loyalty_score_band = (spark.read.format("csv")
      .option("header", "true").option("inferSchema", "true")
      .load('gs://westfield-tom/datalab/Loyalty_indicator/loyalty_score/part-00000-6f33c521-35de-4466-9b8e-aa55c4eb1a15-c000.csv'))

loyalty_score_band.show(5)
loyalty_score_band.count()

print(datetime.datetime.now())

In [ ]:
######################################################################################################################
# Profile Loyalty Indicator

In [53]:
# Customers per Tier
cust_tier=loyalty_score_band.groupBy(col("loyalty_tier")).agg(countDistinct("identifier").alias("Customers")).sort((col("loyalty_tier")))
cust_tier.show()

+------------+--------------------------+
|loyalty_tier|count(DISTINCT identifier)|
+------------+--------------------------+
|  1.Platinum|                    525533|
|      2.Gold|                    503360|
|    3.Silver|                   1097143|
|    4.Bronze|                   2441740|
|5.Occasional|                   7761392|
+------------+--------------------------+



In [57]:
# Customers per score distribution
cust_score_band=loyalty_score_band.withColumn("loyalty_score_band",floor(col('loyalty_score')*10)).groupBy('loyalty_score_band').agg(countDistinct("identifier").alias("Customers")).sort('loyalty_score_band')
cust_score_band.show()

+------------------+---------+
|loyalty_score_band|Customers|
+------------------+---------+
|                 0|  5762805|
|                 1|  1878205|
|                 2|  1711994|
|                 3|   980069|
|                 4|   737141|
|                 5|   480063|
|                 6|   327619|
|                 7|   212452|
|                 8|   136824|
|                 9|    82141|
|                10|    19855|
+------------------+---------+



In [59]:
# Customers per detailed score distribution
cust_detail_score_band=loyalty_score_band.withColumn("loyalty_score_band",floor(col('loyalty_score')*100)).groupBy('loyalty_score_band').agg(countDistinct("identifier").alias("Customers")).sort('loyalty_score_band')
cust_detail_score_band.show(101)

+------------------+---------+
|loyalty_score_band|Customers|
+------------------+---------+
|                 0|  4462788|
|                 1|   289121|
|                 2|   230976|
|                 3|   130184|
|                 4|    88440|
|                 5|    83570|
|                 6|   123918|
|                 7|   132756|
|                 8|   106047|
|                 9|   115005|
|                10|   136276|
|                11|   151284|
|                12|   161012|
|                13|   232747|
|                14|   135956|
|                15|   190129|
|                16|   139698|
|                17|   263948|
|                18|   232760|
|                19|   234395|
|                20|   191952|
|                21|   228246|
|                22|   216414|
|                23|   224505|
|                24|   249729|
|                25|   111822|
|                26|   134453|
|                27|   123671|
|                28|   118895|
|       